## Análisis del laboratorio de masa

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Import custom libraries
import warnings
import sys
import os

# Subir dos niveles desde notebooks/mod_tecnico/ hasta la raíz del repo
repo_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))  
sys.path.append(repo_path)

# Ahora intenta importar
from src.data_processing import ready_to_import

In [4]:
#Importar librerias para graficar
from matplotlib import pyplot as plt
import seaborn as sns

In [5]:
# Importamos lo necesario para trabajar con consultas SQL

from pandasql import sqldf
from pandasql import sqldf

# Definir función de consulta
pysqldf = lambda q: sqldf(q, globals())

#### Se carga el dataset 

In [6]:
# Import data
# Import data from Excel files
masa = pd.read_excel('C:/Users/Miguel Oscar/Projects/data-INSCO-hub/data/raw/masa.xlsx')

In [7]:
# Resumen estadístico de la tabla
masa.describe()

,lab_received_date,scheduled_delivery_date,cleaning_date,calibration_date,delivery_date,delivery_time,assigned_time,piece_count
count,461,454,474,475,480,480.000000,480.000000,480.000000
mean,2025-01-30 01:11:50.629067264,2025-02-18 17:48:53.920704768,2025-01-31 00:57:43.291139328,2025-02-09 17:59:14.526315776,2025-02-15 04:06:00,8.645833,12.291667,5.091667
min,2024-11-06 00:00:00,2024-12-19 00:00:00,2024-03-12 00:00:00,2024-03-13 00:00:00,2025-01-02 00:00:00,0.000000,0.000000,1.000000
25%,2025-01-14 00:00:00,2025-01-31 00:00:00,2025-01-13 00:00:00,2025-01-23 00:00:00,2025-01-24 00:00:00,5.000000,11.000000,1.000000
50%,2025-01-28 00:00:00,2025-02-18 00:00:00,2025-01-29 00:00:00,2025-02-10 00:00:00,2025-02-11 00:00:00,8.000000,13.000000,1.000000
75%,2025-02-25 00:00:00,2025-03-13 00:00:00,2025-02-28 00:00:00,2025-03-08 12:00:00,2025-03-11 00:00:00,11.000000,15.000000,3.000000
max,2025-03-27 00:00:00,2025-04-04 00:00:00,2025-03-28 00:00:00,2025-03-31 00:00:00,2025-03-31 00:00:00,44.000000,30.000000,58.000000
std,NaN,NaN,NaN,NaN,NaN,5.931374,5.242584,8.704613


In [1]:
# Seleccionamos las columnas que contienen fechas y las convertimos a tipo datetime
col_fechas = [col for col in masa.columns if 'date' in col.lower() or 'fecha' in col.lower()]

for col in col_fechas:
    masa[col] = pd.to_datetime(masa[col], errors='coerce', format='%Y/%m/%d')

NameError: name 'masa' is not defined